# CLIP + SAM
- Environment Setting
- CLIP + SAM Evaluation on People Pose
- Hyperparameter Finetuning

## Environment Setting

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install Packages

In [ ]:
!pip install torch opencv-python Pillow
!pip install git+https://github.com/openai/CLIP.git
!pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-1sp50ov5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-1sp50ov5
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 1.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369500 sha256=21d5ec4a3b13a9de2e9cbd8855ca3257161448059dce479fd498bff4073ca161
  Stored in directory: /tmp/pip-ephem-wheel-cache-p6aqoa3a/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.10
    Uninstalling wcwidth-0.2.10:
      Successfully uninstalled wcwidth-0.2.10


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-escjqplo
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-escjqplo
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36588 sha256=34082fd11bd97918f2e2768ed43565ca8bd26ec5d73df9a36ae9d6f9833b09cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-2sa_2x42/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


### Import Packages

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import cv2
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch.jit import Error

# numpy metrics
import os
from tqdm import tqdm

import cv2
from segment_anything import build_sam, SamAutomaticMaskGenerator
from PIL import Image, ImageDraw
import clip

import sys
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

### Image Processing, Data Processing

In [ ]:
# Convert Mask's Boundary Box from XYWH to XYXY format
def convert_box_xywh_to_xyxy(box):
  x1 = box[0]
  y1 = box[1]
  x2 = box[0] + box[2]
  y2 = box[1] + box[3]
  if(box[2]==0 or box[3]==0):
    print(box[2],box[3],[x1, y1, x2, y2])
  return [x1, y1, x2, y2]

# Show Only the Segmented Part in the Given Image
def segment_image(image, segmentation_mask):
    image_array = np.array(image)
    segmented_image_array = np.zeros_like(image_array)
    segmented_image_array[segmentation_mask] = image_array[segmentation_mask]
    segmented_image = Image.fromarray(segmented_image_array)
    black_image = Image.new("RGB", image.size, (0, 0, 0))
    transparency_mask = np.zeros_like(segmentation_mask, dtype=np.uint8)
    transparency_mask[segmentation_mask] = 255
    transparency_mask_image = Image.fromarray(transparency_mask, mode='L')
    black_image.paste(segmented_image, mask=transparency_mask_image)
    return black_image

def gt_to_anns_of_label_mask(mask_gt):
  labels = np.unique(mask_gt)
  anns = []
  for label in labels:
    # skip background
      if label == 0:
          continue
      mask = np.all(mask_gt == label, axis=-1)
      anns.append({
        'area': np.sum(mask),
        'segmentation': mask,
        'label': label,
      })
  return anns

### Retrieve Similarity between Image and Text

In [ ]:
@torch.no_grad()
def retriev(image_features: list, search_text: str) -> int:
    # preprocessed_images = [preprocess(image).to(device) for image in elements]
    tokenized_text = clip.tokenize([search_text]).to(device)
    # stacked_images = torch.stack(preprocessed_images)
    # image_features = model.encode_image(stacked_images)
    text_features = model.encode_text(tokenized_text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    probs = 100. * image_features @ text_features.T
    return probs[:, 0].softmax(dim=0)

### CUDA

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load CLIP

In [ ]:
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 92.8MiB/s]


### Load SAM (Default Params)

In [ ]:
sam_checkpoint = "/content/drive/MyDrive/CSCI567/sam_vit_h_4b8939.pth"
model_type = "vit_h"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam = sam.to(device)

In [ ]:
mask_generator_default = SamAutomaticMaskGenerator(sam)

### Load SAM (Best Params)

### Class Labels

In [ ]:
LABELS = ["Background","Hat","Hair","Glove",
        "Sunglasses","UpperClothes","Dress","Coat","Socks","Pants",
        "Jumpsuits","Scarf","Skirt","Face","Left-arm","Right-arm","Left-leg","Right-leg","Left-shoe","Right-shoe"]

## CLIP + SAM Evaluation on People Pose

In [ ]:
def compute_pix_acc(predicted, target):
    assert predicted.shape == target.shape
    assert len(predicted.shape) == 2
    return (predicted == target).mean()

def compute_IOU(predicted, target):
    assert predicted.shape == target.shape
    assert len(predicted.shape) == 2
    intersection = np.logical_and(target, predicted).sum()
    union = np.logical_or(target, predicted).sum()
    assert union > 0
    return intersection / union
def compute_metric(name, masks, label):
    """ name: data_id
        mask: { label_id: numpy.ndarray(shape=(H, W)) }
        label: np.ndarray(shape=(H, W)) --> numbers from 0 to 19
    """
    pix_acc_metric = { "name": name }
    iou_metric = { "name": name }
    empty = np.zeros_like(label)
    for i, label_name in enumerate(LABELS):
        mask_i = masks.get(i, empty)
        label_i = (label == i)
        if label_i.sum() == 0:
            # pandas dataframe automatically skips nan
            # when computing .count() and .mean()
            iou_metric[label_name] = np.nan
            pix_acc_metric[label_name] = np.nan
        else:
            iou_metric[label_name] = compute_IOU(mask_i, label_i)
            pix_acc_metric[label_name] = compute_pix_acc(mask_i, label_i)

    return iou_metric, pix_acc_metric

In [ ]:
def evaluate(sam_generator, data_cnt=100):
  root = "/content/drive/MyDrive/CSCI567/segment-anything/datasets/people_poses/"
  prompt = "The object of "
  with open(os.path.join(root, f"val_id.txt"), 'r') as lf:
      data_list = [ s.strip() for s in lf.readlines() ]

  try:
    miou_table = []
    pix_acc_table = []
    for data_name in (pbar := tqdm(data_list[:data_cnt])):
      img_path = root +'val_images/' + data_name + '.jpg'
      seg_path = root + 'val_segmentations/' + data_name + '.png'
      # Read Image and Ground truth mask
      image = cv2.imread(img_path)
      if image is None:
          print("\nimage is None", data_name)
          continue
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      mask_gt = cv2.imread(seg_path)
      if mask_gt is None:
          print("\nmask_gt is None", data_name)
          continue

      # Generate masks for all object by SAM
      masks = sam_generator.generate(image)

      # Cut out all masks
      input_img = Image.open(img_path)
      cropped_boxes = []

      for mask in masks:
        crop_box = convert_box_xywh_to_xyxy(mask['bbox'])
        if(crop_box[0]==crop_box[2] or crop_box[1]==crop_box[3]):
          continue
        cropped_boxes.append(segment_image(input_img, mask["segmentation"]).crop(crop_box))

      preprocessed_images = [preprocess(img).to(device) for img in cropped_boxes]
      stacked_images = torch.stack(preprocessed_images)
      image_features = model.encode_image(stacked_images)

      # Get Mask By Label Id
      anns = gt_to_anns_of_label_mask(mask_gt)
      img_miou_sum , img_pixacc_sum, num_class = 0, 0, len(anns)
      predict_masks = {}
      for ann in anns:
        scores = retriev(image_features, prompt+LABELS[ann['label']])
        ## Get Label Index with Highest Score
        predict_idx = np.argmax(scores.cpu())
        predict_idx = predict_idx.cpu()
        predict_masks[ann['label']] = masks[predict_idx]["segmentation"]
      miou, pix_acc = compute_metric(data_name, predict_masks, mask_gt[:,:,0])
      miou_table.append(miou)
      pix_acc_table.append(pix_acc)
    return miou_table, pix_acc_table

  except Exception as e:
    print(e)
    print(miou_table)
    print(pix_acc_table)

def export_csv(miou_table, pix_acc_table, miou_csv_name="random_miou.csv", pix_acc_csv_name="random_pix_acc.csv", export = True):
  miou_table_ = pd.DataFrame(miou_table, columns=miou_table[0].keys()).set_index('name')
  pix_acc_table_ = pd.DataFrame(pix_acc_table, columns=pix_acc_table[0].keys()).set_index('name')
  if export:
    miou_table_.to_csv('/content/drive/MyDrive/CSCI567/segment-anything/notebooks/'+miou_csv_name)
    pix_acc_table_.to_csv('/content/drive/MyDrive/CSCI567/segment-anything/notebooks/'+pix_acc_csv_name)


  # print('miou:\n', miou_table_.mean(axis=None))
  # print('miou per class:\n', miou_table_.mean())
  print()
  # print('pix_acc:\n', pix_acc_table_.mean(axis=None))
  # print('pix_acc per class:\n', pix_acc_table_.mean())
  return miou_table_.mean(), pix_acc_table_.mean()

In [ ]:
# mask_generator_default = SamAutomaticMaskGenerator(
#     sam,
#     points_per_side = 32,
#     points_per_batch = 64,
#     pred_iou_thresh = 0.88,
#     stability_score_thresh = 0.95,
#     stability_score_offset = 1.0,
#     box_nms_thresh = 0.7,
#     crop_n_layers = 0,
#     crop_nms_thresh = 0.7,
#     crop_overlap_ratio = 512 / 1500,
#     crop_n_points_downscale_factor = 1,
#     point_grids = None,
#     min_mask_region_area = 0,
#     output_mode = "binary_mask"
#   )

### Evaluate on Default Sam Generator

In [ ]:
miou_table, pix_acc_table = evaluate(mask_generator_default)

  3%|▎         | 3/100 [00:23<12:30,  7.74s/it]

2 0 [20, 98, 22, 98]


 33%|███▎      | 33/100 [04:08<08:13,  7.37s/it]

0 9 [60, 178, 60, 187]


 56%|█████▌    | 56/100 [06:54<05:04,  6.93s/it]

6 0 [29, 116, 35, 116]
6 0 [29, 116, 35, 116]
6 0 [29, 116, 35, 116]


 77%|███████▋  | 77/100 [09:27<02:48,  7.31s/it]

0 7 [68, 176, 68, 183]
0 7 [68, 176, 68, 183]


100%|██████████| 100/100 [12:12<00:00,  7.32s/it]


In [ ]:
miou_mean_default_100, pixacc_mean_default_100 = export_csv(miou_table, pix_acc_table, miou_csv_name="random_miou_default_100.csv", pix_acc_csv_name="random_pix_acc_default_100.csv")
print('miou per class\n', miou_mean_default_100)


miou per class
 Background      0.000000
Hat             0.317580
Hair            0.264699
Glove           0.091455
Sunglasses      0.001587
UpperClothes    0.203190
Dress           0.000000
Coat            0.344618
Socks           0.076216
Pants           0.371332
Jumpsuits       0.009348
Scarf                NaN
Skirt           0.300369
Face            0.159994
Left-arm        0.172775
Right-arm       0.109500
Left-leg        0.222619
Right-leg       0.110006
Left-shoe       0.137264
Right-shoe      0.104864
dtype: float64


### Evaluate on Other Sams

#### 1: points_per_side

In [ ]:
mask_generator_1 = SamAutomaticMaskGenerator(
    sam,
    points_per_side = 16,
    points_per_batch = 64,
    pred_iou_thresh = 0.88,
    stability_score_thresh = 0.95,
    stability_score_offset = 1.0,
    box_nms_thresh = 0.7,
    crop_n_layers = 0,
    crop_nms_thresh = 0.7,
    crop_overlap_ratio = 512 / 1500,
    crop_n_points_downscale_factor = 1,
    point_grids = None,
    min_mask_region_area = 0,
    output_mode = "binary_mask"
  )

In [ ]:
miou_table_1, pix_acc_table_1 = evaluate(mask_generator_1)

 40%|████      | 40/100 [02:11<03:12,  3.20s/it]

3 0 [52, 144, 55, 144]


 56%|█████▌    | 56/100 [03:02<02:17,  3.11s/it]

0 9 [55, 75, 55, 84]
0 10 [55, 74, 55, 84]


100%|██████████| 100/100 [05:22<00:00,  3.23s/it]


In [ ]:
miou_mean_1_100, pixacc_mean_1_100 = export_csv(miou_table_1, pix_acc_table_1, miou_csv_name="random_miou_1_100.csv", pix_acc_csv_name="random_pix_acc_1_100.csv", export = False)
# print('miou per class\n', miou_mean_1_100)
print("miou per class difference\n", miou_mean_1_100-miou_mean_default_100)


miou per class difference
 Background      0.000000
Hat            -0.031417
Hair           -0.016496
Glove          -0.070638
Sunglasses      0.062702
UpperClothes    0.020409
Dress           0.000000
Coat            0.008805
Socks           0.018756
Pants          -0.034392
Jumpsuits       0.000974
Scarf                NaN
Skirt           0.045313
Face            0.010937
Left-arm       -0.011097
Right-arm       0.026943
Left-leg       -0.012151
Right-leg       0.015227
Left-shoe       0.008581
Right-shoe      0.000299
dtype: float64


#### 2: points_per_side

In [ ]:
mask_generator_2 = SamAutomaticMaskGenerator(
    sam,
    points_per_side = 64,
    points_per_batch = 64,
    pred_iou_thresh = 0.88,
    stability_score_thresh = 0.95,
    stability_score_offset = 1.0,
    box_nms_thresh = 0.7,
    crop_n_layers = 0,
    crop_nms_thresh = 0.7,
    crop_overlap_ratio = 512 / 1500,
    crop_n_points_downscale_factor = 1,
    point_grids = None,
    min_mask_region_area = 0,
    output_mode = "binary_mask"
  )

In [ ]:
miou_table_2, pix_acc_table_2 = evaluate(mask_generator_2)

  0%|          | 0/100 [00:00<?, ?it/s]

6 0 [44, 248, 50, 248]


  3%|▎         | 3/100 [00:59<32:00, 19.80s/it]

2 0 [21, 79, 23, 79]
1 0 [21, 79, 22, 79]
2 0 [21, 79, 23, 79]


  8%|▊         | 8/100 [02:40<31:43, 20.69s/it]

...

 77%|███████▋  | 77/100 [24:54<07:23, 19.28s/it]

0 8 [123, 215, 123, 223]
0 8 [123, 215, 123, 223]
0 7 [68, 176, 68, 183]
0 7 [68, 176, 68, 183]
0 7 [73, 136, 73, 143]
0 7 [73, 136, 73, 143]
0 7 [73, 136, 73, 143]


 96%|█████████▌| 96/100 [30:54<01:17, 19.31s/it]

2 0 [100, 305, 102, 305]
2 0 [100, 305, 102, 305]


100%|██████████| 100/100 [32:08<00:00, 19.28s/it]


In [ ]:
miou_mean_2_100, pixacc_mean_2_100 = export_csv(miou_table_2, pix_acc_table_2, miou_csv_name="random_miou_2_100.csv", pix_acc_csv_name="random_pix_acc_2_100.csv", export = False)
# print('miou per class\n', miou_mean_1_100)
print("miou per class difference\n", miou_mean_2_100-miou_mean_default_100)


miou per class difference
 Background      0.000000
Hat             0.011985
Hair           -0.005930
Glove          -0.059620
Sunglasses      0.023174
UpperClothes   -0.002102
Dress           0.000000
Coat           -0.029023
Socks           0.000088
Pants          -0.043421
Jumpsuits       0.000091
Scarf                NaN
Skirt           0.046035
Face           -0.031783
Left-arm        0.000494
Right-arm       0.010561
Left-leg        0.000318
Right-leg       0.026868
Left-shoe       0.019276
Right-shoe     -0.033610
dtype: float64


#### 3: some thresholds

In [ ]:
mask_generator_3 = SamAutomaticMaskGenerator(
    sam,
    points_per_side = 32,
    points_per_batch = 64,
    pred_iou_thresh = 0.95,
    stability_score_thresh = 0.95,
    stability_score_offset = 1.0,
    box_nms_thresh = 0.9,
    crop_n_layers = 0,
    crop_nms_thresh = 0.9,
    crop_overlap_ratio = 512 / 1500,
    crop_n_points_downscale_factor = 1,
    point_grids = None,
    min_mask_region_area = 0,
    output_mode = "binary_mask"
  )

In [ ]:
miou_table_3, pix_acc_table_3 = evaluate(mask_generator_3)

100%|██████████| 100/100 [15:17<00:00,  9.17s/it]


In [ ]:
miou_mean_3_100, pixacc_mean_3_100 = export_csv(miou_table_3, pix_acc_table_3, miou_csv_name="random_miou_3_100.csv", pix_acc_csv_name="random_pix_acc_3_100.csv", export = False)
# print('miou per class\n', miou_mean_1_100)
print("miou per class difference\n", miou_mean_3_100-miou_mean_default_100)


miou per class difference
 Background      0.000000
Hat             0.020196
Hair           -0.020855
Glove          -0.027154
Sunglasses      0.000000
UpperClothes    0.026187
Dress           0.000000
Coat           -0.016488
Socks          -0.052758
Pants           0.011221
Jumpsuits       0.000000
Scarf                NaN
Skirt           0.000380
Face           -0.011301
Left-arm       -0.080770
Right-arm      -0.004838
Left-leg       -0.078534
Right-leg      -0.017251
Left-shoe      -0.040621
Right-shoe     -0.024334
dtype: float64


#### 4: Hyperparameters in the predictor notebook: https://colab.research.google.com/drive/12yvkr9VomnceYOdQ5dbdoaPIqXRiOPmb

In [ ]:
mask_generator_4 = SamAutomaticMaskGenerator(
    sam,
    points_per_side = 32,
    points_per_batch = 64,
    pred_iou_thresh = 0.86,
    stability_score_thresh = 0.92,
    stability_score_offset = 1.0,
    box_nms_thresh = 0.7,
    crop_n_layers = 1,
    crop_nms_thresh = 0.7,
    crop_overlap_ratio = 512 / 1500,
    crop_n_points_downscale_factor = 2,
    point_grids = None,
    min_mask_region_area = 100,
    output_mode = "binary_mask"
  )

In [ ]:
miou_table_4, pix_acc_table_4 = evaluate(mask_generator_4)

  0%|          | 0/100 [00:00<?, ?it/s]

6 0 [44, 248, 50, 248]


  1%|          | 1/100 [02:50<4:41:56, 170.88s/it]

0 6 [25, 81, 25, 87]


  3%|▎         | 3/100 [03:37<1:30:58, 56.27s/it]

...

 82%|████████▏ | 82/100 [33:24<06:38, 22.16s/it]

0 4 [32, 69, 32, 73]


 96%|█████████▌| 96/100 [38:38<01:30, 22.61s/it]

0 3 [33, 4, 33, 7]


100%|██████████| 100/100 [40:08<00:00, 24.09s/it]


In [ ]:
miou_mean_4_100, pixacc_mean_4_100 = export_csv(miou_table_4, pix_acc_table_4, miou_csv_name="random_miou_4_100.csv", pix_acc_csv_name="random_pix_acc_4_100.csv")
print("miou per class difference\n", miou_mean_4_100-miou_mean_default_100)



miou per class difference
 Background      0.000000
Hat            -0.028639
Hair           -0.039319
Glove          -0.031491
Sunglasses      0.036205
UpperClothes   -0.042745
Dress           0.000000
Coat           -0.001568
Socks           0.009255
Pants          -0.041042
Jumpsuits      -0.007388
Scarf                NaN
Skirt          -0.178598
Face           -0.002003
Left-arm       -0.024950
Right-arm       0.014727
Left-leg        0.046555
Right-leg       0.024230
Left-shoe      -0.028566
Right-shoe      0.014470
dtype: float64
